# Experimental notebook

This is intended for prodding and poking the data structures, not for actual reporting or serious use.
It should definitely, 100% never be trusted for doing anything serious.

In [2]:
import tdt
import collections
import numpy as np
import sys
import seaborn as sns
import matplotlib.pylab as plt
# help(tdt.read_block)


In [3]:
TONE_DURATION = 0.25
IN_TONE_CAPTURE_START_OFFSET = 0.0
IN_TONE_CAPTURE_END_OFFSET = 0.2  # time between start offset and end offset will be counted
OUT_TONE_CAPTURE_START_OFFSET = 0.1
OUT_TONE_CAPTURE_END_OFFSET = 0.25 # time between start offset and end offset will be counted

TONES_PER_SECOND = 2

START_OFFSET = -9.1
END_OFFSET = 9.1

BLOCK_PATH = "/Users/smorgan/Documents/tdttmp/Block-1"
# BLOCK_PATH = "/Volumes/SM Research/Research Data/Simeon Morgan/111_140/2009-07-19 Auditory continuous 10s acclimatisation/2009-07-19 111_140/Block-1"

In [4]:
data = tdt.read_block(BLOCK_PATH, evtype=['epocs', 'snips', 'scalars'], nodata=1)

/Volumes/CaseSensitiveDevPartition/personal/tdt/.venv/lib/python3.8/site-packages/tdt/TDTbin2py.py:792: Warning: tnt file could not be processed
  warnings.warn('tnt file could not be processed', Warning)


read from t=0s to t=7457.32s


In [5]:
filtered = tdt.epoc_filter(data, "TriS", t=[START_OFFSET, END_OFFSET - START_OFFSET])

In [6]:
# Skip the last tone as the end offset should always go a little past the end
tones_to_include = int(((END_OFFSET - START_OFFSET) * TONES_PER_SECOND) - 1)

TIME_RANGE_ONSET_IDX = 0
TIME_RANGE_OFFSET_IDX = 1

SpikeCountRange = collections.namedtuple("SpikeCountRange", ("start", "end"))

all_trials_data = []
cspk_offset = 0
cspk_data = filtered["snips"]["CSPK"]
cspk_length = len(cspk_data["ts"])
np.set_printoptions(threshold=sys.maxsize)

for trial_number in range(len(filtered["time_ranges"][0])):
    in_tone_data = np.zeros((tones_to_include * 2, 32), np.uint)
    #out_tone_data = np.zeros((tones_to_include, 32), np.uint)
    for tone_number, tone_offset in enumerate(filter(lambda x: filtered["time_ranges"][TIME_RANGE_ONSET_IDX][trial_number] < x < filtered["time_ranges"][TIME_RANGE_OFFSET_IDX][trial_number], filtered["epocs"]["StiS"]["onset"])):
        if tone_number >= tones_to_include:
            continue

        spike_cumulator_range = SpikeCountRange(start=tone_offset+IN_TONE_CAPTURE_START_OFFSET, end=tone_offset+IN_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1
            
        spike_cumulator_range = SpikeCountRange(start=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_START_OFFSET, end=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2 + 1, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1


#     print(in_tone_data)
#     plt.rcParams['figure.figsize'] = [100/2.54, 80/2.54]
#     plt.imshow(in_tone_data, cmap='hot', interpolation='nearest')
#     plt.show()
#     ax = sns.heatmap(in_tone_data, linewidth=0.5)
#     plt.show()
#     print("Out tone data")
#     print(out_tone_data)

In [15]:
filtered["epocs"]["StiS"]["offset"][0] - filtered["epocs"]["StiS"]["onset"][0]

0.2500198399999931

In [17]:
filtered["epocs"]["TriS"]

name:	'TriS'
onset:	array([ 121.58359552,  243.58242304,  383.58106112,  509.57984768,
        652.5784576 ,  805.57699072,  951.57558272, 1055.57455872,
       1243.57274624, 1411.57113856, 1576.56953856, 1710.5682432 ,
       1856.56685568, 1964.56580096, 2133.56416   , 2261.56295168,
       2416.56142848, 2548.56015872, 2650.55917056, 2789.5578112 ,
       2942.55635456, 3132.55451648, 3290.55297536, 3460.55139328,
       3583.55016704, 3755.54850816, 3883.54726912, 4073.54544128,
       4232.54391808, 4358.5426944 , 4540.54093824, 4685.53954304,
       4813.53831424, 4971.53678336, 5157.53499648, 5319.53344512,
       5482.53187072, 5588.53083136, 5763.52915456, 5878.52803072,
       5980.5270528 , 6156.52538368, 6338.52360704, 6454.52249088,
       6590.52116992, 6775.5193856 , 6908.51811328, 7091.51635456])
offset:	array([ 131.58356992,  253.5823872 ,  393.58104576,  519.57983232,
        662.57845248,  815.5769856 ,  961.57572096, 1065.57456384,
       1253.57277184, 1421.571123

In [16]:
# start, end = list(zip(filtered["time_ranges"][0], filtered["time_ranges"][1]))[0]

In [7]:
for idx in range(len(filtered["epocs"]["StiS"]["onset"])):
    print("StiS:", filtered["epocs"]["StiS"]["onset"][idx], "Attn:", filtered["epocs"]["Attn"]["onset"][idx], "AFrq:", filtered["epocs"]["AFrq"]["onset"][idx])

StiS: 112.58380288000001 Attn: 112.58374144 AFrq: 112.58374144
StiS: 113.08376064000001 Attn: 113.08368896 AFrq: 113.08368896
StiS: 113.58380032000001 Attn: 113.58372864 AFrq: 113.58372864
StiS: 114.08375808 Attn: 114.08367616 AFrq: 114.08367616
StiS: 114.5837568 Attn: 114.58369536000001 AFrq: 114.58369536000001
StiS: 115.08371456 Attn: 115.08366336 AFrq: 115.08366336
StiS: 115.58375424 Attn: 115.58368256 AFrq: 115.58368256
StiS: 116.083712 Attn: 116.08367104 AFrq: 116.08367104
StiS: 116.58371072 Attn: 116.58366976 AFrq: 116.58366976
StiS: 117.08370944 Attn: 117.083648 AFrq: 117.083648
StiS: 117.58383104 Attn: 117.58374912000001 AFrq: 117.58374912000001
StiS: 118.08370688000001 Attn: 118.0836352 AFrq: 118.0836352
StiS: 118.5837056 Attn: 118.58363392 AFrq: 118.58363392
StiS: 119.08370432 Attn: 119.08363264 AFrq: 119.08363264
StiS: 119.58370304 Attn: 119.58363136 AFrq: 119.58363136
StiS: 120.08370176 Attn: 120.08363008 AFrq: 120.08363008
StiS: 120.58370048 Attn: 120.58361856 AFrq: 120.58